# Question 1

In [2]:
num_nodes = 7

In [3]:
origin = 1

In [4]:
destination = 7

In [5]:
import gurobipy as gp
from gurobipy import GRB

In [6]:
paths,cost = gp.multidict(
{(1, 2): 5,
 (1, 3): 13,
 (1, 5): 45,
 (1, 7): 105,
 (2, 3): 27,
 (2, 4): 19,
 (2, 5): 50,
 (2, 7): 95,
 (3, 4): 14,
 (3, 5): 30,
 (3, 6): 32,
 (4, 3): 14,
 (4, 5): 35,
 (4, 6): 24,
 (5, 4): 35,
 (5, 6): 18,
 (5, 7): 25,
 (6, 4): 24,
 (6, 5): 18,
 (6, 7): 17,
})


In [7]:
mCost = gp.Model('costMin')

Restricted license - for non-production use only - expires 2025-11-24


In [8]:
pathC = mCost.addVars(paths, obj=cost) # pathT is X_ij and time is C_ijS


In [9]:
for i in range(1, num_nodes+1):
    mCost.addConstr(sum(pathC[i,j] for i,j in paths.select(i, '*'))  
                - sum(pathC[j,i] for j,i in paths.select('*',i))
                == (1 if i == origin else -1 if i == destination else 0 ))

In [10]:
# Verify model formulation

mCost.write('shortestpath.lp')

In [11]:
mCost.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 7 rows, 20 columns and 40 nonzeros
Model fingerprint: 0x6b32da78
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 6 rows, 19 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3000000e+01   2.000000e+00   0.000000e+00      0s
       4    6.2000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.200000000e+01


We want to print the variables with a value of greater than 0 because this is the tuple that gurobi's optimization says we should take

In [12]:
print('The path with the least cost is:')
for i,j in paths:
    if(pathC[i,j].x > 0):
        print(i, j, ":", pathC[i,j].x)
print('Total Cost: ' + str(mCost.objVal))

The path with the least time is:
1 3 : 1.0
3 6 : 1.0
6 7 : 1.0
Total Cost: 62.0


# Question 2

## B

In [13]:
supply_demand = dict({'1': -1300,
                      '2': -1200,
                      '3': 300, 
                      '4': 200,
                      '5': 800,
                      '6': 700,
                      '7': 400})

# Create a dictionary to capture shipping costs.

arcs, cost = gp.multidict({
    ('1', '3'): 3,
    ('1', '4'): 4,
    ('2', '3'): 3,
    ('2', '4'): 4,
    ('3', '4'): 7,
    ('3', '5'): 9,
    ('3', '6'): 6,
    ('4', '3'): 7,
    ('4', '6'): 4,
    ('4', '7'): 8,
    ('5', '6'): 3,
    ('6', '7'): 2,
})

In [14]:
mCost = gp.Model('costMin')

In [15]:
pathC = mCost.addVars(arcs, obj=cost) 

In [16]:
flow = mCost.addConstrs((gp.quicksum(pathC.select('*', city)) - gp.quicksum(pathC.select(city, '*')) >= supply_demand[city]
                                  for city in supply_demand.keys()), name="customer")

In [17]:
# Verify model formulation

mCost.write('shortestpath2.lp')

In [18]:
mCost.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0xee94a2cc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 1 rows and 2 columns
Presolve time: 0.01s
Presolved: 6 rows, 10 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.2000000e+03   1.262500e+03   0.000000e+00      0s
       5    2.0900000e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.090000000e+04


In [19]:
print('The path with the least cost is:')
for i,j in arcs:
    if(pathC[i,j].x > 0):
        print(i, j, ":", pathC[i,j].x)
print('Total Cost: ' + str(mCost.objVal))

The path with the least time is:
1 3 : 1100.0
1 4 : 100.0
2 4 : 1200.0
3 5 : 800.0
4 6 : 1100.0
6 7 : 400.0
Total Cost: 20900.0


## C
Suppose that no more than 1,000 units of blood can be transported over any one arc. What is the optimal solution to this revised problem?

In [20]:
mCost.addConstrs((pathC[i,j] <= 1000 for i,j in pathC), name="path_capacity")

{('1', '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('1', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('2', '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('2', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('3', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('3', '5'): <gurobi.Constr *Awaiting Model Update*>,
 ('3', '6'): <gurobi.Constr *Awaiting Model Update*>,
 ('4', '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('4', '6'): <gurobi.Constr *Awaiting Model Update*>,
 ('4', '7'): <gurobi.Constr *Awaiting Model Update*>,
 ('5', '6'): <gurobi.Constr *Awaiting Model Update*>,
 ('6', '7'): <gurobi.Constr *Awaiting Model Update*>}

In [21]:
# Verify model formulation

mCost.write('shortestpath3.lp')

In [22]:
mCost.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 19 rows, 12 columns and 36 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0900000e+04   4.000000e+02   0.000000e+00      0s
       3    2.1000000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.100000000e+04


In [53]:
print('The path with the least cost is:')
for i,j in arcs:
    if(pathC[i,j].x > 0):
        print(i, j, ":", pathC[i,j].x)
print('Total Cost: ' + str(mCost.objVal))

The path with the least cost is:
1 2 : 275.0
1 3 : 650.0
1 4 : 700.0
2 5 : 275.0
3 4 : 25.0
3 5 : 625.0
4 6 : 725.0
5 6 : 900.0
6 1 : 1625.0
Total Cost: 21000.0


# Question 3

In [43]:
arcs, call = gp.multidict({
    (1, 2): 800,
    (1, 3): 650,
    (1, 4): 700,
    (2, 4): 725,
    (2, 5): 700,
    (3, 4): 625,
    (3, 5): 750,
    (4, 6): 725,
    (5, 6): 900,
    (6, 1): GRB.INFINITY
})

In [44]:
mCall = gp.Model('callMax')

In [45]:
pathC = mCall.addVars(arcs) 

In [47]:
mCall.setObjective(pathC[(6, 1)] , GRB.MAXIMIZE)

In [49]:
for i in range(1,7):
    mCall.addConstr(sum(pathC[i,j] for i,j in arcs.select(i, '*'))  
                - sum(pathC[j,i] for j,i in arcs.select('*',i))
                == 0, name='supply_demand')

In [50]:
mCall.addConstrs((pathC[i,j] <= call[i,j] for i,j in pathC), name="path_capacity")

{(1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 6): <gurobi.Constr *Awaiting Model Update*>,
 (5, 6): <gurobi.Constr *Awaiting Model Update*>,
 (6, 1): <gurobi.Constr *Awaiting Model Update*>}

In [51]:
# Verify model formulation

mCall.write('shortestpath4.lp')

In [52]:
mCall.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 16 rows, 10 columns and 30 nonzeros
Model fingerprint: 0xb136dda9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 12 rows and 1 columns
Presolve time: 0.00s
Presolved: 4 rows, 9 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6250000e+03   2.531250e+02   0.000000e+00      0s
       3    1.6250000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.625000000e+03


In [55]:
print('The path with the max flow is:')
for i,j in arcs:
    if(pathC[i,j].x > 0):
        print(i, j, ":", pathC[i,j].x)
print('Total Cost: ' + str(mCall.objVal))

The path with the max flow is:
1 2 : 275.0
1 3 : 650.0
1 4 : 700.0
2 5 : 275.0
3 4 : 25.0
3 5 : 625.0
4 6 : 725.0
5 6 : 900.0
6 1 : 1625.0
Total Cost: 1625.0
